# 🚲 Bergen Bike Availability Prediction System

## AI for Social Good: Sustainable Urban Mobility

**Team Members:** Ang Kar Kin, Nicolas Lasch, Chan Kok Siang

**Course:** 61.501 Production Ready ML (Y2025)

---

### Executive Summary

This project predicts bike availability at bike-sharing stations in Bergen, Norway, enabling:
- **Operators** to proactively manage bike distribution
- **Users** to find the best station based on location and predicted availability

We use interpretable ML models (Random Forest & XGBoost) to forecast hourly trip counts per station.

---

## 1. Environment Setup & Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import joblib
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

In [ ]:
np.random.seed(42)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

---

## 2. Data Loading & Preprocessing Functions

In [ ]:
def load_dataset(filepath):
    return pd.read_csv(filepath)


def parse_timestamps(df):
    df = df.copy()
    df['start_time'] = pd.to_datetime(df['start_time'])
    df['end_time'] = pd.to_datetime(df['end_time'])
    return df


def extract_temporal_features(df):
    df = df.copy()
    df['hour'] = df['start_time'].dt.hour
    df['day'] = df['start_time'].dt.day
    df['month'] = df['start_time'].dt.month
    df['dayofweek'] = df['start_time'].dt.dayofweek
    df['is_rush_hour'] = df['hour'].isin([7, 8, 9, 16, 17, 18]).astype(int)
    return df


def clean_missing_values(df):
    df = df.copy()
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())
    return df


def remove_outliers(df, column, multiplier=3.0):
    df = df.copy()
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - multiplier * iqr
    upper = q3 + multiplier * iqr
    mask = (df[column] >= lower) & (df[column] <= upper)
    return df[mask]


def preprocess_pipeline(filepath):
    df = load_dataset(filepath)
    df = parse_timestamps(df)
    df = extract_temporal_features(df)
    df = clean_missing_values(df)
    df = remove_outliers(df, 'duration')
    return df

---

## 3. Feature Engineering Functions

In [ ]:
def aggregate_station_hourly(df):
    hourly = df.groupby([
        'start_station_id',
        'start_station_name',
        df['start_time'].dt.date.rename('date'),
        'hour'
    ]).agg({
        'duration': 'count',
        'temperature': 'mean',
        'max_temperature': 'mean',
        'min_temperature': 'mean',
        'wind_speed': 'mean',
        'precipitation': 'mean',
        'humidity': 'mean',
        'sunshine': 'mean',
        'season': 'first',
        'is_holiday': 'first',
        'is_weekend': 'first',
        'start_station_latitude': 'first',
        'start_station_longitude': 'first',
        'is_rush_hour': 'first',
        'dayofweek': 'first',
        'month': 'first'
    }).reset_index()
    
    hourly = hourly.rename(columns={'duration': 'trip_count'})
    return hourly


def create_lag_features(df, lags=[1, 2, 3, 24]):
    df = df.copy()
    df = df.sort_values(['start_station_id', 'date', 'hour'])
    
    for lag in lags:
        df[f'trip_count_lag_{lag}'] = df.groupby('start_station_id')['trip_count'].shift(lag)
    
    return df


def create_rolling_features(df, windows=[3, 6, 24]):
    df = df.copy()
    df = df.sort_values(['start_station_id', 'date', 'hour'])
    
    for window in windows:
        df[f'trip_count_rolling_{window}h'] = df.groupby('start_station_id')['trip_count'].transform(
            lambda x: x.rolling(window, min_periods=1).mean()
        )
    
    return df


def create_weather_interactions(df):
    df = df.copy()
    df['temp_humidity'] = df['temperature'] * df['humidity']
    df['temp_wind'] = df['temperature'] * df['wind_speed']
    df['rain_wind'] = df['precipitation'] * df['wind_speed']
    df['is_rainy'] = (df['precipitation'] > 0).astype(int)
    df['is_hot'] = (df['temperature'] > 20).astype(int)
    df['is_cold'] = (df['temperature'] < 5).astype(int)
    return df


def encode_cyclic_features(df):
    df = df.copy()
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
    df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    return df


def calculate_station_popularity(df):
    df = df.copy()
    station_stats = df.groupby('start_station_id')['trip_count'].agg(['mean', 'std']).reset_index()
    station_stats.columns = ['start_station_id', 'station_avg_trips', 'station_std_trips']
    df = df.merge(station_stats, on='start_station_id', how='left')
    return df


def feature_engineering_pipeline(df):
    df = aggregate_station_hourly(df)
    df = create_lag_features(df)
    df = create_rolling_features(df)
    df = create_weather_interactions(df)
    df = encode_cyclic_features(df)
    df = calculate_station_popularity(df)
    df = df.dropna()
    return df


def prepare_train_test_split(df, test_ratio=0.2):
    df = df.sort_values(['date', 'hour'])
    split_idx = int(len(df) * (1 - test_ratio))
    train_df = df.iloc[:split_idx].copy()
    test_df = df.iloc[split_idx:].copy()
    return train_df, test_df


def get_feature_columns():
    return [
        'temperature', 'max_temperature', 'min_temperature',
        'wind_speed', 'precipitation', 'humidity', 'sunshine',
        'season', 'is_holiday', 'is_weekend', 'is_rush_hour',
        'hour_sin', 'hour_cos', 'dayofweek_sin', 'dayofweek_cos',
        'month_sin', 'month_cos',
        'trip_count_lag_1', 'trip_count_lag_2', 'trip_count_lag_3', 'trip_count_lag_24',
        'trip_count_rolling_3h', 'trip_count_rolling_6h', 'trip_count_rolling_24h',
        'temp_humidity', 'temp_wind', 'rain_wind',
        'is_rainy', 'is_hot', 'is_cold',
        'station_avg_trips', 'station_std_trips',
        'start_station_latitude', 'start_station_longitude'
    ]

---

## 4. Model Training & Evaluation Functions

In [ ]:
def train_random_forest(X_train, y_train, params=None):
    if params is None:
        params = {
            'n_estimators': 100,
            'max_depth': 15,
            'min_samples_split': 5,
            'min_samples_leaf': 2,
            'random_state': 42,
            'n_jobs': -1
        }
    model = RandomForestRegressor(**params)
    model.fit(X_train, y_train)
    return model


def train_xgboost(X_train, y_train, params=None):
    if params is None:
        params = {
            'n_estimators': 100,
            'max_depth': 8,
            'learning_rate': 0.1,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'random_state': 42,
            'n_jobs': -1
        }
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train)
    return model


def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    return {'RMSE': rmse, 'MAE': mae, 'R2': r2}


def get_feature_importance(model, feature_names):
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    return importance_df


def hyperparameter_tuning_rf(X_train, y_train, X_val, y_val):
    param_grid = [
        {'n_estimators': 50, 'max_depth': 10},
        {'n_estimators': 100, 'max_depth': 15},
        {'n_estimators': 150, 'max_depth': 20},
        {'n_estimators': 200, 'max_depth': 25}
    ]
    
    best_score = float('inf')
    best_model = None
    best_params = None
    
    for params in param_grid:
        model = train_random_forest(X_train, y_train, {**params, 'random_state': 42, 'n_jobs': -1})
        metrics = evaluate_model(model, X_val, y_val)
        if metrics['RMSE'] < best_score:
            best_score = metrics['RMSE']
            best_model = model
            best_params = params
    
    return best_model, best_params


def hyperparameter_tuning_xgb(X_train, y_train, X_val, y_val):
    param_grid = [
        {'n_estimators': 50, 'max_depth': 6, 'learning_rate': 0.1},
        {'n_estimators': 100, 'max_depth': 8, 'learning_rate': 0.1},
        {'n_estimators': 100, 'max_depth': 10, 'learning_rate': 0.05},
        {'n_estimators': 150, 'max_depth': 12, 'learning_rate': 0.05}
    ]
    
    best_score = float('inf')
    best_model = None
    best_params = None
    
    for params in param_grid:
        model = train_xgboost(X_train, y_train, {**params, 'random_state': 42, 'n_jobs': -1})
        metrics = evaluate_model(model, X_val, y_val)
        if metrics['RMSE'] < best_score:
            best_score = metrics['RMSE']
            best_model = model
            best_params = params
    
    return best_model, best_params


def compare_models(models, X_test, y_test):
    results = []
    for name, model in models.items():
        metrics = evaluate_model(model, X_test, y_test)
        metrics['model'] = name
        results.append(metrics)
    return pd.DataFrame(results).set_index('model')

---

## 5. Station Recommendation Functions

In [ ]:
def calculate_distance(lat1, lon1, lat2, lon2):
    from math import radians, cos, sin, asin, sqrt
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371
    return c * r


def get_nearby_stations(user_lat, user_lon, stations_df, radius_km=2.0):
    stations = stations_df.copy()
    stations['distance_km'] = stations.apply(
        lambda row: calculate_distance(user_lat, user_lon, row['start_station_latitude'], row['start_station_longitude']),
        axis=1
    )
    nearby = stations[stations['distance_km'] <= radius_km].sort_values('distance_km')
    return nearby


def calculate_recommendation_score(distance, predicted_trips, max_distance=2.0):
    distance_score = 1 - (distance / max_distance)
    availability_score = min(predicted_trips / 10, 1.0)
    combined_score = 0.4 * distance_score + 0.6 * availability_score
    return combined_score


def recommend_best_station(user_lat, user_lon, target_hour, stations_info, features_df, model, feature_cols, top_k=3):
    unique_stations = stations_info[['start_station_id', 'start_station_name', 
                                      'start_station_latitude', 'start_station_longitude']].drop_duplicates()
    nearby = get_nearby_stations(user_lat, user_lon, unique_stations, radius_km=2.0)
    
    if nearby.empty:
        return pd.DataFrame()
    
    recommendations = []
    for _, station in nearby.iterrows():
        station_features = features_df[
            (features_df['start_station_id'] == station['start_station_id']) &
            (features_df['hour'] == target_hour)
        ]
        
        if station_features.empty:
            continue
        
        latest_features = station_features.iloc[-1:][feature_cols]
        predicted_trips = model.predict(latest_features)[0]
        score = calculate_recommendation_score(station['distance_km'], predicted_trips)
        
        recommendations.append({
            'station_id': station['start_station_id'],
            'station_name': station['start_station_name'],
            'latitude': station['start_station_latitude'],
            'longitude': station['start_station_longitude'],
            'distance_km': station['distance_km'],
            'predicted_trips': predicted_trips,
            'recommendation_score': score
        })
    
    rec_df = pd.DataFrame(recommendations)
    rec_df = rec_df.sort_values('recommendation_score', ascending=False).head(top_k)
    return rec_df


def format_recommendation_output(recommendations):
    if recommendations.empty:
        return "No stations found nearby. Try increasing the search radius."
    
    output_lines = ["Top Station Recommendations:\n"]
    for idx, row in recommendations.iterrows():
        output_lines.append(f"Station: {row['station_name']}")
        output_lines.append(f"  Distance: {row['distance_km']:.2f} km")
        output_lines.append(f"  Predicted Activity: {row['predicted_trips']:.1f} trips/hour")
        output_lines.append(f"  Score: {row['recommendation_score']:.3f}")
        output_lines.append("")
    return "\n".join(output_lines)

---

## 6. Visualization Functions

In [ ]:
def set_plot_style():
    plt.style.use('seaborn-v0_8-whitegrid')
    sns.set_palette("husl")
    plt.rcParams['figure.figsize'] = (12, 6)
    plt.rcParams['font.size'] = 12


def plot_feature_importance(importance_df, top_n=15):
    set_plot_style()
    fig, ax = plt.subplots(figsize=(10, 8))
    top_features = importance_df.head(top_n)
    bars = ax.barh(range(len(top_features)), top_features['importance'].values)
    ax.set_yticks(range(len(top_features)))
    ax.set_yticklabels(top_features['feature'].values)
    ax.set_xlabel('Feature Importance')
    ax.set_title(f'Top {top_n} Most Important Features')
    ax.invert_yaxis()
    for bar in bars:
        width = bar.get_width()
        ax.text(width, bar.get_y() + bar.get_height()/2, f'{width:.4f}', ha='left', va='center')
    plt.tight_layout()
    return fig


def plot_actual_vs_predicted(y_true, y_pred):
    set_plot_style()
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.scatter(y_true, y_pred, alpha=0.5, s=10)
    min_val = min(y_true.min(), y_pred.min())
    max_val = max(y_true.max(), y_pred.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')
    ax.set_xlabel('Actual Trip Count')
    ax.set_ylabel('Predicted Trip Count')
    ax.set_title('Actual vs Predicted Trip Counts')
    ax.legend()
    plt.tight_layout()
    return fig


def plot_residuals(y_true, y_pred):
    set_plot_style()
    residuals = y_true - y_pred
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    axes[0].scatter(y_pred, residuals, alpha=0.5, s=10)
    axes[0].axhline(y=0, color='r', linestyle='--', lw=2)
    axes[0].set_xlabel('Predicted Values')
    axes[0].set_ylabel('Residuals')
    axes[0].set_title('Residuals vs Predicted Values')
    axes[1].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
    axes[1].axvline(x=0, color='r', linestyle='--', lw=2)
    axes[1].set_xlabel('Residual Value')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Distribution of Residuals')
    plt.tight_layout()
    return fig


def plot_hourly_patterns(df):
    set_plot_style()
    hourly_avg = df.groupby('hour')['trip_count'].mean()
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(hourly_avg.index, hourly_avg.values, marker='o', linewidth=2, markersize=8)
    ax.fill_between(hourly_avg.index, hourly_avg.values, alpha=0.3)
    ax.set_xlabel('Hour of Day')
    ax.set_ylabel('Average Trip Count')
    ax.set_title('Average Bike Trips by Hour of Day')
    ax.set_xticks(range(24))
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    return fig


def plot_weather_correlation(df):
    set_plot_style()
    weather_cols = ['temperature', 'humidity', 'wind_speed', 'precipitation', 'sunshine', 'trip_count']
    corr_matrix = df[weather_cols].corr()
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, square=True, 
                linewidths=1, cbar_kws={"shrink": 0.8}, ax=ax, fmt='.2f')
    ax.set_title('Correlation Matrix: Weather vs Trip Count')
    plt.tight_layout()
    return fig


def plot_model_comparison(comparison_df):
    set_plot_style()
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    metrics = ['RMSE', 'MAE', 'R2']
    colors = sns.color_palette("husl", len(comparison_df))
    for idx, metric in enumerate(metrics):
        bars = axes[idx].bar(comparison_df.index, comparison_df[metric], color=colors)
        axes[idx].set_title(f'Model Comparison: {metric}')
        axes[idx].set_ylabel(metric)
        axes[idx].tick_params(axis='x', rotation=45)
        for bar in bars:
            height = bar.get_height()
            axes[idx].text(bar.get_x() + bar.get_width()/2., height,
                          f'{height:.4f}', ha='center', va='bottom')
    plt.tight_layout()
    return fig


def plot_station_map(stations_df, recommendations=None, user_location=None):
    set_plot_style()
    fig, ax = plt.subplots(figsize=(12, 10))
    ax.scatter(stations_df['start_station_longitude'], stations_df['start_station_latitude'], 
               c='gray', s=20, alpha=0.5, label='All Stations')
    if recommendations is not None and not recommendations.empty:
        ax.scatter(recommendations['longitude'], recommendations['latitude'],
                  c='green', s=100, marker='*', label='Recommended Stations', zorder=5)
        for _, rec in recommendations.iterrows():
            ax.annotate(f"{rec['station_name'][:20]}...", 
                       (rec['longitude'], rec['latitude']),
                       xytext=(5, 5), textcoords='offset points', fontsize=8)
    if user_location:
        ax.scatter(user_location[1], user_location[0], c='red', s=150, 
                  marker='X', label='Your Location', zorder=6)
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Bike Stations Map')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    return fig


def plot_seasonal_patterns(df):
    set_plot_style()
    season_map = {0: 'Spring', 1: 'Summer', 2: 'Fall', 3: 'Winter'}
    df_temp = df.copy()
    df_temp['season_name'] = df_temp['season'].map(season_map)
    seasonal_avg = df_temp.groupby('season_name')['trip_count'].mean()
    fig, ax = plt.subplots(figsize=(10, 6))
    colors = ['#90EE90', '#FFD700', '#FFA500', '#87CEEB']
    bars = ax.bar(seasonal_avg.index, seasonal_avg.values, color=colors, edgecolor='black')
    ax.set_xlabel('Season')
    ax.set_ylabel('Average Trip Count')
    ax.set_title('Average Bike Trips by Season')
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
               f'{height:.2f}', ha='center', va='bottom')
    plt.tight_layout()
    return fig

---

## 7. Load and Explore Data

**Note:** Download the dataset from Kaggle and update the path below:
https://www.kaggle.com/datasets/amykzhang/bergen-bike-sharing-dataset-2023

In [ ]:
DATA_PATH = 'bergen_merged.csv'

raw_df = pd.read_csv(DATA_PATH)
print(f"Dataset Shape: {raw_df.shape}")
print(f"Total Records: {raw_df.shape[0]:,}")
print(f"Features: {raw_df.shape[1]}")

In [ ]:
print("Dataset Info:")
print("=" * 50)
raw_df.info()

In [ ]:
print("First 5 Records:")
raw_df.head()

In [ ]:
print("Statistical Summary:")
raw_df.describe()

In [ ]:
print("Missing Values:")
missing = raw_df.isnull().sum()
missing_pct = (missing / len(raw_df) * 100).round(2)
missing_df = pd.DataFrame({'Count': missing, 'Percentage': missing_pct})
missing_df[missing_df['Count'] > 0]

In [ ]:
print(f"Unique Stations: {raw_df['start_station_id'].nunique()}")
print(f"Date Range: {raw_df['start_time'].min()} to {raw_df['start_time'].max()}")
print(f"Average Trip Duration: {raw_df['duration'].mean():.2f} seconds")

---

## 8. Data Preprocessing

In [ ]:
print("Starting preprocessing pipeline...")
processed_df = preprocess_pipeline(DATA_PATH)

print(f"\nOriginal Records: {len(raw_df):,}")
print(f"After Preprocessing: {len(processed_df):,}")
print(f"Records Removed: {len(raw_df) - len(processed_df):,}")

In [ ]:
print("New Temporal Features:")
processed_df[['start_time', 'hour', 'day', 'month', 'dayofweek', 'is_rush_hour']].head(10)

---

## 9. Exploratory Data Analysis

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

hourly_trips = processed_df.groupby('hour').size()
axes[0, 0].bar(hourly_trips.index, hourly_trips.values, color='steelblue', edgecolor='black')
axes[0, 0].set_title('Total Trips by Hour of Day')
axes[0, 0].set_xlabel('Hour')
axes[0, 0].set_ylabel('Number of Trips')
axes[0, 0].set_xticks(range(0, 24, 2))

daily_trips = processed_df.groupby('dayofweek').size()
day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
axes[0, 1].bar(range(7), daily_trips.values, color='forestgreen', edgecolor='black')
axes[0, 1].set_title('Total Trips by Day of Week')
axes[0, 1].set_xlabel('Day')
axes[0, 1].set_ylabel('Number of Trips')
axes[0, 1].set_xticks(range(7))
axes[0, 1].set_xticklabels(day_names)

axes[1, 0].scatter(processed_df['temperature'], processed_df['duration'], alpha=0.1, s=1)
axes[1, 0].set_title('Temperature vs Trip Duration')
axes[1, 0].set_xlabel('Temperature (°C)')
axes[1, 0].set_ylabel('Duration (seconds)')

season_map = {0: 'Spring', 1: 'Summer', 2: 'Fall', 3: 'Winter'}
season_trips = processed_df['season'].map(season_map).value_counts()
colors = ['#90EE90', '#FFD700', '#FFA500', '#87CEEB']
axes[1, 1].pie(season_trips.values, labels=season_trips.index, colors=colors, autopct='%1.1f%%', startangle=90)
axes[1, 1].set_title('Trips by Season')

plt.tight_layout()
plt.show()

In [ ]:
top_stations = processed_df['start_station_name'].value_counts().head(10)

fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.barh(range(len(top_stations)), top_stations.values, color='coral', edgecolor='black')
ax.set_yticks(range(len(top_stations)))
ax.set_yticklabels(top_stations.index)
ax.set_xlabel('Number of Trips')
ax.set_title('Top 10 Most Popular Stations')
ax.invert_yaxis()

for bar in bars:
    width = bar.get_width()
    ax.text(width, bar.get_y() + bar.get_height()/2, f'{int(width):,}', ha='left', va='center')

plt.tight_layout()
plt.show()

---

## 10. Feature Engineering

In [ ]:
print("Starting feature engineering pipeline...")
features_df = feature_engineering_pipeline(processed_df)

print(f"\nFeature DataFrame Shape: {features_df.shape}")
print(f"Total Samples (station-hour combinations): {len(features_df):,}")

In [ ]:
print("Feature Columns:")
feature_cols = get_feature_columns()
for i, col in enumerate(feature_cols, 1):
    print(f"{i:2}. {col}")

In [ ]:
print("Sample of Engineered Features:")
features_df[['start_station_name', 'date', 'hour', 'trip_count', 
             'trip_count_lag_1', 'trip_count_rolling_3h', 'temperature', 
             'is_rush_hour', 'station_avg_trips']].head(10)

In [ ]:
plot_weather_correlation(features_df)
plt.show()

In [ ]:
plot_hourly_patterns(features_df)
plt.show()

In [ ]:
plot_seasonal_patterns(features_df)
plt.show()

---

## 11. Train-Validation-Test Split

In [ ]:
train_val_df, test_df = prepare_train_test_split(features_df, test_ratio=0.2)
train_df, val_df = prepare_train_test_split(train_val_df, test_ratio=0.25)

print(f"Training Set: {len(train_df):,} samples")
print(f"Validation Set: {len(val_df):,} samples")
print(f"Test Set: {len(test_df):,} samples")
print(f"\nTotal: {len(train_df) + len(val_df) + len(test_df):,} samples")

In [ ]:
X_train = train_df[feature_cols]
y_train = train_df['trip_count']

X_val = val_df[feature_cols]
y_val = val_df['trip_count']

X_test = test_df[feature_cols]
y_test = test_df['trip_count']

print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"X_test shape: {X_test.shape}")

---

## 12. Model Training

In [ ]:
print("Training baseline Random Forest...")
rf_baseline = train_random_forest(X_train, y_train)

rf_baseline_metrics = evaluate_model(rf_baseline, X_val, y_val)
print("\nBaseline Random Forest Performance (Validation):")
for metric, value in rf_baseline_metrics.items():
    print(f"  {metric}: {value:.4f}")

In [ ]:
print("Training baseline XGBoost...")
xgb_baseline = train_xgboost(X_train, y_train)

xgb_baseline_metrics = evaluate_model(xgb_baseline, X_val, y_val)
print("\nBaseline XGBoost Performance (Validation):")
for metric, value in xgb_baseline_metrics.items():
    print(f"  {metric}: {value:.4f}")

---

## 13. Hyperparameter Tuning

In [ ]:
print("Tuning Random Forest hyperparameters...")
rf_tuned, rf_best_params = hyperparameter_tuning_rf(X_train, y_train, X_val, y_val)

print(f"\nBest Random Forest Parameters:")
for param, value in rf_best_params.items():
    print(f"  {param}: {value}")

rf_tuned_metrics = evaluate_model(rf_tuned, X_val, y_val)
print("\nTuned Random Forest Performance (Validation):")
for metric, value in rf_tuned_metrics.items():
    print(f"  {metric}: {value:.4f}")

In [ ]:
print("Tuning XGBoost hyperparameters...")
xgb_tuned, xgb_best_params = hyperparameter_tuning_xgb(X_train, y_train, X_val, y_val)

print(f"\nBest XGBoost Parameters:")
for param, value in xgb_best_params.items():
    print(f"  {param}: {value}")

xgb_tuned_metrics = evaluate_model(xgb_tuned, X_val, y_val)
print("\nTuned XGBoost Performance (Validation):")
for metric, value in xgb_tuned_metrics.items():
    print(f"  {metric}: {value:.4f}")

---

## 14. Model Comparison & Final Evaluation

In [ ]:
models = {
    'RF_Baseline': rf_baseline,
    'RF_Tuned': rf_tuned,
    'XGB_Baseline': xgb_baseline,
    'XGB_Tuned': xgb_tuned
}

comparison_results = compare_models(models, X_test, y_test)
print("Model Comparison on Test Set:")
print("=" * 50)
comparison_results

In [ ]:
plot_model_comparison(comparison_results)
plt.show()

In [ ]:
best_model_name = comparison_results['RMSE'].idxmin()
best_model = models[best_model_name]

print(f"Best Model: {best_model_name}")
print(f"Test RMSE: {comparison_results.loc[best_model_name, 'RMSE']:.4f}")
print(f"Test MAE: {comparison_results.loc[best_model_name, 'MAE']:.4f}")
print(f"Test R²: {comparison_results.loc[best_model_name, 'R2']:.4f}")

---

## 15. Feature Importance Analysis

In [ ]:
importance_df = get_feature_importance(best_model, feature_cols)
print("Top 15 Most Important Features:")
importance_df.head(15)

In [ ]:
plot_feature_importance(importance_df, top_n=15)
plt.show()

---

## 16. Prediction Diagnostics

In [ ]:
y_pred_test = best_model.predict(X_test)

plot_actual_vs_predicted(y_test.values, y_pred_test)
plt.show()

In [ ]:
plot_residuals(y_test.values, y_pred_test)
plt.show()

In [ ]:
errors = np.abs(y_test.values - y_pred_test)
error_percentiles = np.percentile(errors, [25, 50, 75, 90, 95, 99])

print("Error Distribution:")
print(f"  25th percentile: {error_percentiles[0]:.2f} trips")
print(f"  50th percentile (median): {error_percentiles[1]:.2f} trips")
print(f"  75th percentile: {error_percentiles[2]:.2f} trips")
print(f"  90th percentile: {error_percentiles[3]:.2f} trips")
print(f"  95th percentile: {error_percentiles[4]:.2f} trips")
print(f"  99th percentile: {error_percentiles[5]:.2f} trips")

---

## 17. Station Recommendation System

In [ ]:
user_latitude = 60.3913
user_longitude = 5.3221
target_hour = 8

print(f"User Location: ({user_latitude}, {user_longitude})")
print(f"Target Hour: {target_hour}:00")

In [ ]:
recommendations = recommend_best_station(
    user_lat=user_latitude,
    user_lon=user_longitude,
    target_hour=target_hour,
    stations_info=features_df,
    features_df=features_df,
    model=best_model,
    feature_cols=feature_cols,
    top_k=5
)

print(format_recommendation_output(recommendations))

In [ ]:
unique_stations = features_df[['start_station_id', 'start_station_name', 
                                'start_station_latitude', 'start_station_longitude']].drop_duplicates()

plot_station_map(unique_stations, recommendations, 
                 user_location=(user_latitude, user_longitude))
plt.show()

---

## 18. Model Failure Analysis

In [ ]:
test_df_analysis = test_df.copy()
test_df_analysis['prediction'] = y_pred_test
test_df_analysis['absolute_error'] = np.abs(test_df_analysis['trip_count'] - test_df_analysis['prediction'])

high_error_threshold = np.percentile(test_df_analysis['absolute_error'], 95)
high_error_cases = test_df_analysis[test_df_analysis['absolute_error'] > high_error_threshold]

print(f"High Error Cases (top 5% errors, threshold: {high_error_threshold:.2f}):")
print(f"Number of cases: {len(high_error_cases)}")
print("\nSample of high-error predictions:")
high_error_cases[['start_station_name', 'date', 'hour', 'trip_count', 'prediction', 
                  'absolute_error', 'temperature', 'precipitation', 'is_rush_hour']].head(10)

In [ ]:
print("Analysis of High Error Cases:")
print(f"  Average actual trips: {high_error_cases['trip_count'].mean():.2f}")
print(f"  Average predicted trips: {high_error_cases['prediction'].mean():.2f}")
print(f"  Rush hour percentage: {high_error_cases['is_rush_hour'].mean()*100:.1f}%")
print(f"  Rainy conditions percentage: {(high_error_cases['precipitation'] > 0).mean()*100:.1f}%")
print(f"  Holiday percentage: {high_error_cases['is_holiday'].mean()*100:.1f}%")

---

## 19. Hypothesis Testing

In [ ]:
rush_hour_trips = features_df[features_df['is_rush_hour'] == 1]['trip_count'].mean()
non_rush_trips = features_df[features_df['is_rush_hour'] == 0]['trip_count'].mean()

print("HYPOTHESIS 1: Higher activity during rush hours")
print(f"  Average trips during rush hours: {rush_hour_trips:.2f}")
print(f"  Average trips during non-rush hours: {non_rush_trips:.2f}")
print(f"  Difference: {rush_hour_trips - non_rush_trips:.2f} more trips during rush hours")
print(f"  Result: {'CONFIRMED' if rush_hour_trips > non_rush_trips else 'NOT CONFIRMED'}")

In [ ]:
temp_correlation = features_df[['temperature', 'trip_count']].corr().iloc[0, 1]

print("\nHYPOTHESIS 2: Temperature affects bike rentals")
print(f"  Correlation between temperature and trips: {temp_correlation:.4f}")
print(f"  Result: {'CONFIRMED' if abs(temp_correlation) > 0.1 else 'NOT CONFIRMED'}")

In [ ]:
rainy_trips = features_df[features_df['precipitation'] > 0]['trip_count'].mean()
dry_trips = features_df[features_df['precipitation'] == 0]['trip_count'].mean()

print("\nHYPOTHESIS 3: Rain reduces bike rentals")
print(f"  Average trips during rain: {rainy_trips:.2f}")
print(f"  Average trips when dry: {dry_trips:.2f}")
print(f"  Reduction: {((dry_trips - rainy_trips) / dry_trips * 100):.1f}%")
print(f"  Result: {'CONFIRMED' if rainy_trips < dry_trips else 'NOT CONFIRMED'}")

In [ ]:
weekday_trips = features_df[features_df['is_weekend'] == 0]['trip_count'].mean()
weekend_trips = features_df[features_df['is_weekend'] == 1]['trip_count'].mean()

print("\nHYPOTHESIS 4: Weekend patterns differ from weekdays")
print(f"  Average weekday trips: {weekday_trips:.2f}")
print(f"  Average weekend trips: {weekend_trips:.2f}")
print(f"  Difference: {abs(weekday_trips - weekend_trips):.2f} trips")
print(f"  Result: {'CONFIRMED' if abs(weekday_trips - weekend_trips) > 0.5 else 'NOT CONFIRMED'}")

---

## 20. Save Model (Optional)

In [ ]:
joblib.dump(best_model, f'{best_model_name}.joblib')
print(f"Best model saved as {best_model_name}.joblib")

comparison_results.to_csv('model_comparison_results.csv')
print("Model comparison results saved as model_comparison_results.csv")

---

## 21. Conclusions & Recommendations

### Key Findings:

1. **Model Performance**: Our best model achieves strong predictive performance with temporal and weather features
2. **Important Features**: Lag features and rolling averages are crucial predictors
3. **Weather Impact**: Temperature and precipitation significantly affect bike usage
4. **Temporal Patterns**: Clear daily and seasonal patterns exist in bike sharing

### Recommendations for Operators:

1. **Pre-position bikes** during predicted low-availability periods
2. **Weather-based planning** for bike redistribution
3. **Rush hour management** with proactive rebalancing

### Future Improvements:

1. Include real-time station capacity data
2. Add event-based features (concerts, sports events)
3. Integrate traffic and public transport data
4. Develop a mobile app for real-time recommendations

---

## Pipeline Diagram

```
┌─────────────┐    ┌──────────────┐    ┌───────────────┐
│   Raw Data  │ -> │  Preprocess  │ -> │   Feature     │
│   Loading   │    │  (Clean,     │    │  Engineering  │
│             │    │   Parse)     │    │  (Lags, Rolls)│
└─────────────┘    └──────────────┘    └───────────────┘
                                              │
                                              v
┌─────────────┐    ┌──────────────┐    ┌───────────────┐
│   Model     │ <- │    Model     │ <- │  Train/Val/   │
│   Saving    │    │   Training   │    │  Test Split   │
│             │    │  (RF, XGB)   │    │               │
└─────────────┘    └──────────────┘    └───────────────┘
                          │
                          v
┌─────────────┐    ┌──────────────┐    ┌───────────────┐
│  Station    │ <- │   Predict    │ <- │   Evaluate    │
│  Recommend  │    │   Trips/Hour │    │  (RMSE, MAE)  │
│             │    │              │    │               │
└─────────────┘    └──────────────┘    └───────────────┘
```

In [ ]:
print("="*60)
print("PROJECT COMPLETED SUCCESSFULLY!")
print("="*60)
print(f"Total features engineered: {len(feature_cols)}")
print(f"Best performing model: {best_model_name}")
print(f"Final Test RMSE: {comparison_results.loc[best_model_name, 'RMSE']:.4f}")
print(f"Final Test MAE: {comparison_results.loc[best_model_name, 'MAE']:.4f}")
print(f"Final Test R²: {comparison_results.loc[best_model_name, 'R2']:.4f}")